In [1]:
# modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import chardet
import csv
import os
import matplotlib.dates as mdates
from IPython.display import Image

## Inserez les chemins vers le fichiers CSV:

In [2]:
# on récupère les informations du dossier
dossier = './raw_data'
contenuDossier = os.listdir(dossier)
print(contenuDossier)
nombrePoints = 0
nomPoints = []
numeroPoints = []
# on parcours le dossier en recherchant les fichiers des points
for x in contenuDossier:
    if x.startswith('point'):
        nombrePoints += 1
        nomPoints.append(x)
        numeroPoints.append(x[5:7])
print(numeroPoints)

['point14_29_07_15', 'point15_29_07_15', '.DS_Store', 'schema_dispositif_AvAv_23_07_15.png', 'point13_29_07_15']
['14', '15', '13']


In [3]:
def read_csv (chemin_fichier):
    #Detecter separateur
    with open(chemin_fichier, 'r') as file:
        sniffer = csv.Sniffer()
        sample_data = file.read(1024)
        detecter_separateur = (sniffer.sniff(sample_data).delimiter)

    if "Titre" in open(chemin_fichier).readline():
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur, skiprows=1)
    else:
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur)

    return data_frame 

In [4]:
# on crée un dictionnaire avec toutes les informations pour chaque point et on met les dictionnaires dans une liste
data = []
for x in nomPoints :
    dico = {}
    # le numéro
    dico['numero'] = x[5:7]
    # le nom
    dico['nom'] = x
    # la date
    dico['date'] = x[8:18]
    # le chemin vers le dossier
    dico['chemin'] = dossier + '/' + x
    with open(dico['chemin'] + '/geometrie.txt', 'r') as fichier:
        lignes = fichier.readlines()
        # le nom du capteur
        dico['capteur'] = lignes[1][:4]
        # la profondeur du capteur
        if lignes[3] == '\n' :
            dico['profondeur'] = 0
        else : 
            dico['profondeur'] = lignes[3]
        # les profondeurs auquelles on fait les mesures
        dico['profondeurMesures'] = []
        profMes = lignes[5].split(';')
        for num in profMes :
            dico['profondeurMesures'].append(int(num))
    # on va ensuite mettre les dataframes dans le dictionnaire
    contenuDossierPoint = os.listdir(dico['chemin'])
    for x in contenuDossierPoint:
        # la pression
        if x.startswith('p') and x.endswith('.csv'):
            dico['pression'] = read_csv(dico['chemin'] + '/' + x)
        # la température
        if x.startswith('t') and x.endswith('.csv'):
            dico['temperature'] = read_csv(dico['chemin'] + '/' + x)
    # dans les configurations du capteur on va chercher les données
    with open('configuration/pressure_sensors/P' + '505' +'.csv' ) as fichier :
        # normalement il faut mettre dico['capteur'][1:] pas '505' mais là on a pas les fichiers :/
        lignes = fichier.readlines()
        for l in lignes :
            if l.startswith('Intercept') :
                dico['intercept'] = float(l[10:-2])
            if l.startswith('dU/dH') :
                dico['dU/dH'] = float(l[6:-2])
            if l.startswith('dU/dT') :
                dico['dU/dT'] = float(l[6:-2]) 
    data.append(dico)


In [37]:
# en faisant data[0] on a accès à toutes les informations du point 1
data[0]

{'numero': '14',
 'nom': 'point14_29_07_15',
 'date': '29_07_15',
 'chemin': './raw_data/point14_29_07_15',
 'capteur': 'p507',
 'profondeur': 0,
 'profondeurMesures': [10, 20, 30, 40],
 'pression':                   dates  tension  temperature_stream        dH
 6   2015-07-23 15:30:00  1.13336              17.344  0.218805
 7   2015-07-23 15:45:00  1.23682              17.319  0.147703
 8   2015-07-23 16:00:00  1.23663              17.320  0.147842
 9   2015-07-23 16:15:00  1.20249              17.315  0.171178
 10  2015-07-23 16:30:00  1.22831              17.357  0.153896
 ..                  ...      ...                 ...       ...
 558 2015-07-29 09:30:00  1.23991              15.562  0.128738
 559 2015-07-29 09:45:00  1.23709              15.541  0.130468
 560 2015-07-29 10:00:00  1.24273              15.513  0.126337
 561 2015-07-29 10:15:00  1.24022              15.599  0.128881
 562 2015-07-29 10:30:00  1.30335              19.928  0.127152
 
 [557 rows x 4 columns],
 'tempe

### On modifie les nom des colonnes du dataframe

In [6]:
# pour les températures
for x in data :
    # on supprime la première colonne si elle ne sert à rien
    for i in x['temperature'].columns :
        if not( i.startswith('Date') or i.startswith('Temp')) :
            del x['temperature'][i]
    for i in x['pression'].columns :
        if not( i.startswith('Date') or i.startswith('Tension') or i.startswith('Temp')) :
            del x['pression'][i]
    # on renomme les colonnes
    profondeur = x['profondeurMesures']
    colonnesT = ['dates']
    for num in profondeur :
        colonnesT.append('Temp_profondeur_'+ str(num))
    x['temperature'].columns = colonnesT
    colonnesP = ['dates', 'tension', 'temperature_stream']
    x['pression'].columns = colonnesP
    

In [7]:
# notamment la température et la pression
data[0]['temperature'].head(5)

,dates,Temp_profondeur_10,Temp_profondeur_20,Temp_profondeur_30,Temp_profondeur_40
0,07/23/15 02:00:00 PM,26.207,27.554,26.378,28.097
1,07/23/15 02:15:00 PM,28.147,27.087,26.867,29.265
2,07/23/15 02:30:00 PM,28.419,28.543,27.431,29.615
3,07/23/15 02:45:00 PM,18.390,21.223,18.319,19.555
4,07/23/15 03:00:00 PM,18.509,21.318,18.461,19.413


### Création du fichier `info.txt`

In [8]:
# Ouvrez un fichier en mode écriture ('w')
for x in data :
    if os.path.exists(x['chemin'] + '/point' + x['numero'] + '_info.txt') :
        os.remove(x['chemin'] + '/point' + x['numero'] + '_info.txt')
    with open(x['chemin'] + '/point' + x['numero'] + '_info.txt', 'w') as fichier:
        # Parcourez le dictionnaire et écrivez les données dans le fichier
        fichier.write('Point_Name,Point' + x['numero'] + '\n')
        fichier.write('P_Sensor_Name,' + x['capteur'] + '\n')
        fichier.write('Implantation_Date,' + x['date'] + '\n')
        fichier.write('Meas_date, \n')
        fichier.write('River_Deb,' + '\n')
        fichier.write('Delta_h,' + '\n')


In [9]:
data[0]['pression']

,dates,tension,temperature_stream
0,07/23/15 02:00:00 PM,0.01183,45.230
1,07/23/15 02:15:00 PM,0.00080,21.979
2,07/23/15 02:30:00 PM,0.00080,21.609
3,07/23/15 02:45:00 PM,0.00092,21.910
4,07/23/15 03:00:00 PM,0.00088,20.966
...,...,...,...
558,07/29/15 09:30:00 AM,1.23991,15.562
559,07/29/15 09:45:00 AM,1.23709,15.541
560,07/29/15 10:00:00 AM,1.24273,15.513
561,07/29/15 10:15:00 AM,1.24022,15.599


## Conversion de valeurs de tension a differance de charge

In [53]:
for point in data:
    k0 = float(point['intercept'])
    k1 = float(point['dU/dH'])
    k2 = float(point['dU/dT'])
    
    # Calcul de la colonne 'dH' en utilisant la formule H = 1/k1 * (U - k0 - k2 * T)
    U = point['pression']['tension'].astype(float)
    T = point['pression']['temperature_stream'].astype(float)
    point['pression']['dH'] = (1 / k1) * (U - k0 - k2 * T)

data[0]['pression']

/var/folders/6k/xvw3sqcs3212vgmrjm186p4m0000gn/T/ipykernel_42414/3049274173.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point['pression']['dH'] = (1 / k1) * (U - k0 - k2 * T)


,dates,tension,temperature_stream,dH
6,2015-07-23 15:30:00,1.13336,17.344,0.025003
7,2015-07-23 15:45:00,1.23682,17.319,0.013416
8,2015-07-23 16:00:00,1.23663,17.320,0.013436
9,2015-07-23 16:15:00,1.20249,17.315,0.017280
10,2015-07-23 16:30:00,1.22831,17.357,0.014314
...,...,...,...,...
558,2015-07-29 09:30:00,1.23991,15.562,0.015771
559,2015-07-29 09:45:00,1.23709,15.541,0.016120
560,2015-07-29 10:00:00,1.24273,15.513,0.015530
561,2015-07-29 10:15:00,1.24022,15.599,0.015679


la suite est à adapter

In [12]:
# Conversion dans le bon format date
for point in data:
    point['pression']['dates'] = pd.to_datetime(point['pression']['dates'], format='%m/%d/%y %I:%M:%S %p')
    point['temperature']['dates'] = pd.to_datetime(point['temperature']['dates'], format='%m/%d/%y %I:%M:%S %p')

In [13]:
data[0]['temperature'].head(5)

,dates,Temp_profondeur_10,Temp_profondeur_20,Temp_profondeur_30,Temp_profondeur_40
0,2015-07-23 14:00:00,26.207,27.554,26.378,28.097
1,2015-07-23 14:15:00,28.147,27.087,26.867,29.265
2,2015-07-23 14:30:00,28.419,28.543,27.431,29.615
3,2015-07-23 14:45:00,18.390,21.223,18.319,19.555
4,2015-07-23 15:00:00,18.509,21.318,18.461,19.413


In [14]:
def nombre_de_valeurs_manquantes(capteur):
    # Compte des valeurs manquantes (NaN) pour chaque colonne
    nombre_de_valeurs_manquantes = capteur.isna().sum()
    return nombre_de_valeurs_manquantes

# Exemple d'utilisation :
nombre_valeurs_manquantes_pression = nombre_de_valeurs_manquantes(point['pression'])
nombre_valeurs_manquantes_temperature = nombre_de_valeurs_manquantes(point['temperature'])

print("Nombre de valeurs manquantes pour la pression :")
print(nombre_valeurs_manquantes_pression)

print("\nNombre de valeurs manquantes pour la température :")
print(nombre_valeurs_manquantes_temperature)

Nombre de valeurs manquantes pour la pression :
dates                 0
tension               0
temperature_stream    0
dH                    0
dtype: int64

Nombre de valeurs manquantes pour la température :
dates                 0
Temp_profondeur_10    0
Temp_profondeur_20    0
Temp_profondeur_30    0
Temp_profondeur_40    0
dtype: int64


In [15]:
from scipy import stats
# Traiter chaque colonne sauf la colonne des dates
columns_to_process = [col for col in data[0]['pression'].columns if col != "dates"]

# Créer un DataFrame vide pour stocker les données traitées
df_processed2 = data[0]['pression'][["dates"]].copy()

# Boucler à travers chaque colonne à traiter
for column_name in columns_to_process:
    # Calculer le Z-score pour la colonne
    z_scores = np.abs(stats.zscore(data[0]['pression'][column_name]))
    
    # Définir un seuil pour le Z-score (par exemple, 3)
    threshold = 3
    # Sélectionner les lignes avec des Z-scores inférieurs au seuil
    df_cleaned = data[0]['pression'][z_scores < threshold]
    
    # Copier les données traitées dans df_processed
    df_processed2[column_name] = df_cleaned[column_name]

# Afficher les statistiques des données traitées
print(df_processed2.describe())

                               dates     tension  temperature_stream  \
count                            563  557.000000          558.000000   
mean   2015-07-26 12:14:59.999999744    1.260235           16.267339   
min              2015-07-23 14:00:00    1.133360           15.372000   
25%              2015-07-25 01:07:30    1.236630           15.799000   
50%              2015-07-26 12:15:00    1.253720           16.159000   
75%              2015-07-27 23:22:30    1.275460           16.687000   
max              2015-07-29 10:30:00    1.342260           19.928000   
std                              NaN    0.031142            0.584637   

               dH  
count  557.000000  
mean     0.121561  
min      0.073489  
25%      0.108556  
50%      0.124671  
75%      0.134989  
max      0.218805  
std      0.019947  


## Supprimer les valeurs aberrantes

In [16]:
from scipy import stats

# Fonction pour détecter et supprimer les valeurs aberrantes en utilisant le Z-score
def remove_outliers_z_score(df, col_name, threshold=3):
    mean_1 = np.mean(df[col_name])
    std_1  = np.std (df[col_name])
    z_scores = np.abs((df[col_name] - mean_1)/std_1)

    # z_scores = np.abs(stats.zscore(df[col_name]))
    df_cleaned = df[(z_scores < threshold)]
    return df_cleaned

In [17]:
data_processed = data.copy()

In [18]:
data_processed[0]['pression'] = remove_outliers_z_score(data[0]['pression'], 'dH')

In [19]:
data[0]['temperature']

,dates,Temp_profondeur_10,Temp_profondeur_20,Temp_profondeur_30,Temp_profondeur_40
0,2015-07-23 14:00:00,26.207,27.554,26.378,28.097
1,2015-07-23 14:15:00,28.147,27.087,26.867,29.265
2,2015-07-23 14:30:00,28.419,28.543,27.431,29.615
3,2015-07-23 14:45:00,18.390,21.223,18.319,19.555
4,2015-07-23 15:00:00,18.509,21.318,18.461,19.413
...,...,...,...,...,...
559,2015-07-29 09:45:00,15.986,15.772,15.867,14.625
560,2015-07-29 10:00:00,15.986,15.724,15.867,14.577
561,2015-07-29 10:15:00,15.986,15.748,15.867,14.745
562,2015-07-29 10:30:00,18.081,15.629,18.461,17.320
